In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.image as mpimg
%matplotlib inline

In [3]:

from keras.layers import Dense, Flatten, Activation, MaxPooling2D, Lambda
from keras.models import Sequential
from keras.layers.core import Lambda
from keras.layers.convolutional import Convolution2D
from keras.optimizers import SGD, Adam, RMSprop

Using TensorFlow backend.


In [4]:
# generator

# read the csv file
dataframe = pd.read_csv('data\driving_log.csv')

NameError: name 'X_data' is not defined

In [5]:
dataframe.head()

,center,left,right,steering,throttle,brake,speed
0,IMG/center_2016_12_01_13_30_48_287.jpg,IMG/left_2016_12_01_13_30_48_287.jpg,IMG/right_2016_12_01_13_30_48_287.jpg,0.0,0.0,0.0,22.148290
1,IMG/center_2016_12_01_13_30_48_404.jpg,IMG/left_2016_12_01_13_30_48_404.jpg,IMG/right_2016_12_01_13_30_48_404.jpg,0.0,0.0,0.0,21.879630
2,IMG/center_2016_12_01_13_31_12_937.jpg,IMG/left_2016_12_01_13_31_12_937.jpg,IMG/right_2016_12_01_13_31_12_937.jpg,0.0,0.0,0.0,1.453011
3,IMG/center_2016_12_01_13_31_13_037.jpg,IMG/left_2016_12_01_13_31_13_037.jpg,IMG/right_2016_12_01_13_31_13_037.jpg,0.0,0.0,0.0,1.438419
4,IMG/center_2016_12_01_13_31_13_177.jpg,IMG/left_2016_12_01_13_31_13_177.jpg,IMG/right_2016_12_01_13_31_13_177.jpg,0.0,0.0,0.0,1.418236


In [ ]:
def generator(X_data, y_data):
    
    num_samples = len(data)
    
    for offset in range(0, num_samples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
    
    yield batch_x, batch_y
    
batch_x, batch_y = next(generator(X_data, y_data))